TimeSeries Basics
=================

TimeSeries are largely inspired by Matlab's `timeseries` and `tscollection`. Every TimeSeries contains the following attributes:

- `time`: A numpy array that contains the time vector.
- `data`: A dict where each entry is a numpy array, with the first dimension corresponding to time.
- `events`: An optional list of events.
- `time_info`: Metadata corresponding to time, that contains at least the time unit.
- `data_info`: Optional metadata.

Time and Data
-------------

A TimeSeries in its simplest form contains a time vector and at least one data series. For example:

In [ ]:
import kineticstoolkit.lab as ktk
import numpy as np

ts = ktk.TimeSeries()
ts.time = np.arange(0, 10, 0.1)  # 10 seconds at 10 Hz
ts.data['Sinus'] = np.sin(ts.time)

ts

In [ ]:
ts.data

TimeSeries can be [plotted](../api/kineticstoolkit.TimeSeries.plot.rst) directly using Matplotlib:

In [ ]:
ts.plot()

A TimeSeries can contain many independent data that share a same time vector:

In [ ]:
ts.data['Cosinus'] = np.cos(ts.time)

ts.data

In [ ]:
ts.plot()

A TimeSeries can also contain multidimensional data, as long as the first dimension corresponds to time.


For the rest of this tutorial, we will load a TimeSeries that contains forces and moments during manual wheelchair propulsion. These forces and moments are expressed as Nx4 series of vectors:

\begin{equation}
\text{Forces} = \begin{bmatrix}
F_x(0) & F_y(0) & F_z(0) & 0 \\
F_x(1) & F_y(1) & F_z(1) & 0 \\
F_x(2) & F_y(2) & F_z(2) & 0 \\
... & ... & ... & ... \\
F_x(N-1) & F_y(N-1) & F_z(N-1) & 0
\end{bmatrix}
\end{equation}

\begin{equation}
\text{Moments} = \begin{bmatrix}
M_x(0) & M_y(0) & M_z(0) & 0 \\
M_x(1) & M_y(1) & M_z(1) & 0 \\
M_x(2) & M_y(2) & M_z(2) & 0 \\
... & ... & ... & ... \\
M_x(N-1) & M_y(N-1) & M_z(N-1) & 0
\end{bmatrix}
\end{equation}

In [ ]:
ts = ktk.load(ktk.config.root_folder + '/data/timeseries/smartwheel.ktk.zip')

ts

In [ ]:
ts.data

In [ ]:
ts.plot()

Metadata
--------

The `time_info` property associates metadata to the time vector. It is a dictionary where each key is the name of one metadata. By default, `time_info` includes the `Unit` metadata, which corresponds to `s`. Any other metadata can be added by adding new keys in `time_info`.

In [ ]:
ts.time_info

Similarly, the `data_info` property associates metadata to data. This property is a dictionary of dictionaries, where the outer key corresponds to the data key, and the inner key is the metadata. The [TimeSeries.add_data_info()](../api/kineticstoolkit.TimeSeries.add_data_info.rst) method eases the management of `data_info`.

In [ ]:
ts = ts.add_data_info('Forces', 'Unit', 'N')
ts = ts.add_data_info('Moments', 'Unit', 'Nm')

ts.data_info

In [ ]:
ts.data_info['Forces']

Unless explicitly mentioned, metadata is not used for calculation and is optional. It is simply a way to clarify the data by adding information to it. Some functions however read metadata: for example, the [TimeSeries.plot()](../api/kineticstoolkit.TimeSeries.plot.rst) method looks for possible `Unit` metadata and prints it on the y axis.

In [ ]:
ts.plot()

Events
------

In the figure above, we see that the TimeSeries contains cyclic data that could be characterized by events. A first spike was generated at about 4 seconds: this event corresponds to a synchronization signal that we generated by gently impacting the instrumented pushrim. Thereafter, we see a series of pushes and recoveries. We will add these events to the TimeSeries.

There are several ways to edit the events of a TimeSeries:
- Editing events manually, using the [TimeSeries.add_event()](../api/kineticstoolkit.TimeSeries.add_event.rst) and [TimeSeries.remove_event()](../api/kineticstoolkit.TimeSeries.remove_event.rst) methods;
- Editing events interactively, using the [TimeSeries.ui_edit_events()](../api/kineticstoolkit.TimeSeries.ui_edit_events.rst) method;
- Adding events automatically, for example using the [cycles](../cycles.rst) module that can detect cycles automatically.

In this tutorial, we will add the events manually.

In [ ]:
ts = ts.add_event(4.35, 'sync')
ts = ts.add_event(8.56, 'push')
ts = ts.add_event(9.93, 'recovery')
ts = ts.add_event(10.50, 'push')
ts = ts.add_event(11.12, 'recovery')
ts = ts.add_event(11.78, 'push')
ts = ts.add_event(12.33, 'recovery')
ts = ts.add_event(13.39, 'push')
ts = ts.add_event(13.88, 'recovery')
ts = ts.add_event(14.86, 'push')
ts = ts.add_event(15.30, 'recovery')

These 11 events are now added to the TimeSeries' list of events:

In [ ]:
ts

In [ ]:
ts.events

If we plot again the TimeSeries, we can see the added events.

In [ ]:
ts.plot()

### Using events to synchronize TimeSeries ###

Let's see how we can make use of these events. First, the `sync` event may be set to the time origin, which could be useful to sync this TimeSeries with other instrumentation. This can be done using the [TimeSeries.sync_event()](../api/kineticstoolkit.TimeSeries.sync_event.rst) function, which shifts the TimeSeries' time and every event's time accordingly.

In [ ]:
ts = ts.sync_event('sync')
ts.plot()